In [6]:
import numpy as np
import os
import urllib
import gzip
import struct


def read_data(label_url, image_url):
    with gzip.open(label_url) as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        label = np.fromstring(flbl.read(), dtype=np.int8)
    with gzip.open(image_url, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        image = np.fromstring(fimg.read(), dtype=np.uint8).reshape(len(label), rows, cols)
    return (label, image)

path='/home/ec2-user/MNIST_data/'
(train_lbl, train_img) = read_data(
    path+'train-labels-idx1-ubyte.gz', path+'train-images-idx3-ubyte.gz')
(val_lbl, val_img) = read_data(
    path+'t10k-labels-idx1-ubyte.gz', path+'t10k-images-idx3-ubyte.gz')



In [7]:

# mxnet related
import mxnet as mx

def to4d(img):
    return img.reshape(img.shape[0], 1, 28, 28).astype(np.float32)/255

batch_size = 100
train_iter = mx.io.NDArrayIter(to4d(train_img), train_lbl, batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(to4d(val_img), val_lbl, batch_size)

#
# Create a place holder variable for the input data
data = mx.sym.Variable('data')
# Flatten the data from 4-D shape (batch_size, num_channel, width, height)
# into 2-D (batch_size, num_channel*width*height)
data = mx.sym.Flatten(data=data)

# The first fully-connected layer
fc1  = mx.sym.FullyConnected(data=data, name='fc1', num_hidden=128)
# Apply relu to the output of the first fully-connnected layer
act1 = mx.sym.Activation(data=fc1, name='relu1', act_type="relu")

# The second fully-connected layer and the according activation function
fc2  = mx.sym.FullyConnected(data=act1, name='fc2', num_hidden = 64)
act2 = mx.sym.Activation(data=fc2, name='relu2', act_type="relu")

# The thrid fully-connected layer, note that the hidden size should be 10, which is the number of unique digits
fc3  = mx.sym.FullyConnected(data=act2, name='fc3', num_hidden=10)
# The softmax and loss layer
mlp  = mx.sym.SoftmaxOutput(data=fc3, name='softmax')

# We visualize the network structure with output size (the batch_size is ignored.)
shape = {"data" : (batch_size, 1, 28, 28)}
# mx.viz.plot_network(symbol=mlp, shape=shape)



In [8]:

# @@@ AUTOTEST_OUTPUT_IGNORED_CELL
import logging
logging.getLogger().setLevel(logging.DEBUG)

model = mx.model.FeedForward(
    symbol = mlp,       # network structure
    num_epoch = 10,     # number of data passes for training
    learning_rate = 0.1 # learning rate of SGD
)
model.fit(
    X=train_iter,       # training data
    eval_data=val_iter, # validation data
    batch_end_callback = mx.callback.Speedometer(batch_size, 200) # output progress for each 200 data batches
)

# @@@ AUTOTEST_OUTPUT_IGNORED_CELL
# plt.imshow(val_img[0], cmap='Greys_r')
# plt.axis('off')
# plt.show()
prob = model.predict(val_img[0:1].astype(np.float32)/255)[0]
assert max(prob) > 0.99, "Low prediction accuracy."
print 'Classified as %d with probability %f' % (prob.argmax(), max(prob))


INFO:root:Start training with [cpu(0)]
INFO:root:Epoch[0] Batch [200]	Speed: 47780.37 samples/sec	Train-accuracy=0.111350
INFO:root:Epoch[0] Batch [400]	Speed: 47553.04 samples/sec	Train-accuracy=0.110400
INFO:root:Epoch[0] Batch [600]	Speed: 47619.85 samples/sec	Train-accuracy=0.139400
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=1.301
INFO:root:Epoch[0] Validation-accuracy=0.272200
INFO:root:Epoch[1] Batch [200]	Speed: 48014.83 samples/sec	Train-accuracy=0.421350
INFO:root:Epoch[1] Batch [400]	Speed: 47791.66 samples/sec	Train-accuracy=0.747900
INFO:root:Epoch[1] Batch [600]	Speed: 47732.92 samples/sec	Train-accuracy=0.833550
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=1.261
INFO:root:Epoch[1] Validation-accuracy=0.840400
INFO:root:Epoch[2] Batch [200]	Speed: 47859.83 samples/sec	Train-accuracy=0.863050
INFO:root:Epoch[2] Batch [400]	Speed: 47700.71 samples/sec	Train-accuracy=0.885100
INFO:root:Epoch[2] Batch [600]	Speed: 47507.2

Classified as 7 with probability 0.999215
